In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.2 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=2aa06c0f3346056b4196dbe42337c2591b8f3648809a764bf3252311b22bc1ae
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.7 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_UK
 a0529-a0611_US		       Github
 ccc_compiled.csv	       heatmap529_UK.ipynb
 csv			       heatmap530.ipynb
 csv.zip		       Heat_Map_Online_language.ipynb
 dataframe_csv		       __MACOSX
 dataframe_csv_EU	       mapping_UK
 dataframe_csv_UK	       mapping_US
 dataframe_result_529_UK.csv   non_violant1.csv
 dataframe_result_530_UK.csv   non_violant2.csv
 dataframe_result_531_UK.csv   UK
 dataframe_result_csv	       violant.csv
 dataframe_result_csv_EU      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_601 = 'csv/US06_01.csv'

In [8]:
ct = pd.read_csv(ct_path_601)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,96623207231_10158378197842232,Facebook,2020-06-01 22:49:27,2021-08-31 16:30:44,link,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,WATCH LIVE: President Trump is addressing unre...,1.0,http://on.nbc10.com/b4bKlRO,https://www.facebook.com/96623207231/posts/101...,691512,-1.134969,1.0,35,4,77,15,0,3,1,28,0,1,33,19,61,10,8,15,14,25,0,4,3535,NBC10 Philadelphia,nbcphiladelphia,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NBC10 Philadelphia is the place to go for excl...,True,2009-06-29 01:05:54,en,0,3535|10158378197842232
1,1265297223574474_2731921486912033,Facebook,2020-06-01 22:49:25,2021-04-14 09:17:59,photo,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,UPDATE: Family Memorial Service for George Flo...,1.0,https://www.facebook.com/ThePressRelease4u/pho...,https://www.facebook.com/1265297223574474/post...,477,-24.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,1,0,1,2,0,0,5972645,The Press Release with Portia Bright,ThePressRelease4u,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Live interviews and real news facts all consum...,False,2017-10-15 01:53:17,en,0,5972645|2731921486912033
2,100395078149910_161877328668351,Facebook,2020-06-01 22:49:24,2021-06-18 22:28:53,photo,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,How many of you were at Saturday’s event “Stan...,1.0,https://www.facebook.com/Topekaunited/photos/a...,https://www.facebook.com/100395078149910/posts...,0,4.736842,1.0,48,8,3,30,0,1,0,0,0,7,4,4,2,4,0,0,0,0,0,1,12642648,Topeka United,Topekaunited,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,We are a group of individuals who came togethe...,False,2019-12-25 15:29:15,en,0,12642648|161877328668351
3,762995300426539_3094901393902573,Facebook,2020-06-01 22:49:24,2021-06-13 14:30:17,live_video_complete,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,LIVE: @NBCNews Special Report: President Trump...,RIGHT NOW:,2.0,https://www.facebook.com/NBCNews/videos/130791...,https://www.facebook.com/762995300426539/posts...,3153,-5.666667,2.0,7,1,1,1,0,1,1,0,0,0,31,4,17,9,2,1,4,0,0,0,2186839,Shannon Miller,Topekaunited,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Weekend AM Anchor/Reporter for NBC Connecticut...,False,2014-10-17 16:10:07,en,0,2186839|3094901393902573
4,33957807912_10157309663472913,Facebook,2020-06-01 22:49:22,2021-05-17 12:46:14,status,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,LIVE: @NBCNews Special Report: President Trump...,What we witnessed happening to George Floyd ha...,2.0,https://www.facebook.com/NBCNews/videos/130791...,https://www.facebook.com/33957807912/posts/101...,2353,1.323529,2.0,27,2,7,9,0,0,0,0,0,0,18,2,5,9,0,0,0,0,0,0,8050974,Lamplighters Music Theatre,Lamplighters,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,http://lamplighters.org/about/reviews.html h...,False,2008-07-17 18:14:18,en,0,8050974|10157309663472913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32484,166280891785_10157007368411786,Facebook,2020-06-01 15:15:05,2021-08-30 23:

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{},{}
1,{},{}
2,{},{}
3,{},{}
4,{},{}


In [24]:
ct.to_csv('a0601.csv')

In [25]:
ct=pd.read_csv('a0601.csv')

In [26]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0529.csv		       Github
 a0530.csv		       heatmap529_UK.ipynb
 a0601.csv		       heatmap530.ipynb
 ccc_compiled.csv	       Heat_Map_Online_language.ipynb
 csv			       __MACOSX
 csv.zip		       mapping_UK
 dataframe_csv		       mapping_US
 dataframe_csv_EU	       non_violant1.csv
 dataframe_csv_UK	       non_violant2.csv
 dataframe_result_529_UK.csv   UK
 dataframe_result_530_UK.csv   violant.csv
 dataframe_result_531_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_csv


In [27]:
###
#ct=pd.read_csv('a0601.csv')

In [28]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,96623207231_10158378197842232,Facebook,2020-06-01 22:49:27,2021-08-31 16:30:44,link,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,WATCH LIVE: President Trump is addressing unre...,1.0,http://on.nbc10.com/b4bKlRO,https://www.facebook.com/96623207231/posts/101...,691512,-1.134969,1.0,35,4,77,15,0,3,1,28,0,1,33,19,61,10,8,15,14,25,0,4,3535,NBC10 Philadelphia,nbcphiladelphia,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NBC10 Philadelphia is the place to go for excl...,True,2009-06-29 01:05:54,en,0,3535|10158378197842232,{},{},{}
1,1,1265297223574474_2731921486912033,Facebook,2020-06-01 22:49:25,2021-04-14 09:17:59,photo,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,UPDATE: Family Memorial Service for George Flo...,1.0,https://www.facebook.com/ThePressRelease4u/pho...,https://www.facebook.com/1265297223574474/post...,477,-24.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,1,0,1,2,0,0,5972645,The Press Release with Portia Bright,ThePressRelease4u,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Live interviews and real news facts all consum...,False,2017-10-15 01:53:17,en,0,5972645|2731921486912033,{},{},{}
2,2,100395078149910_161877328668351,Facebook,2020-06-01 22:49:24,2021-06-18 22:28:53,photo,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,How many of you were at Saturday’s event “Stan...,1.0,https://www.facebook.com/Topekaunited/photos/a...,https://www.facebook.com/100395078149910/posts...,0,4.736842,1.0,48,8,3,30,0,1,0,0,0,7,4,4,2,4,0,0,0,0,0,1,12642648,Topeka United,Topekaunited,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,We are a group of individuals who came togethe...,False,2019-12-25 15:29:15,en,0,12642648|161877328668351,{},{},{}
3,3,762995300426539_3094901393902573,Facebook,2020-06-01 22:49:24,2021-06-13 14:30:17,live_video_complete,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,LIVE: @NBCNews Special Report: President Trump...,RIGHT NOW:,2.0,https://www.facebook.com/NBCNews/videos/130791...,https://www.facebook.com/762995300426539/posts...,3153,-5.666667,2.0,7,1,1,1,0,1,1,0,0,0,31,4,17,9,2,1,4,0,0,0,2186839,Shannon Miller,Topekaunited,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Weekend AM Anchor/Reporter for NBC Connecticut...,False,2014-10-17 16:10:07,en,0,2186839|3094901393902573,{},{},{}
4,4,33957807912_10157309663472913,Facebook,2020-06-01 22:49:22,2021-05-17 12:46:14,status,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,LIVE: @NBCNews Special Report: President Trump...,What we witnessed happening to George Floyd ha...,2.0,https://www.facebook.com/NBCNews/videos/130791...,https://www.facebook.com/33957807912/posts/101...,2353,1.323529,2.0,27,2,7,9,0,0,0,0,0,0,18,2,5,9,0,0,0,0,0,0,8050974,Lamplighters Music Theatre,Lamplighters,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,http://lamplighters.org/about/reviews.html h...,False,2008-07-17 18:14:18,en,0,8050974|10157309663472913,{},{},{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [29]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [30]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [31]:
###
ct_ner = ct

In [32]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{},{}
1,{},{},{}
2,{},{},{}
3,{},{},{}
4,{},{},{}


In [33]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [34]:
ct_ner['NER-msg']

0                   []
1                   []
2                   []
3                   []
4                   []
             ...      
32484    [Minneapolis]
32485     [New Mexico]
32486               []
32487               []
32488               []
Name: NER-msg, Length: 32489, dtype: object

In [35]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [36]:
ct_ner['NER']

0                                                       []
1                                                       []
2                                                       []
3                                                       []
4                                                       []
                               ...                        
32484                                        [Minneapolis]
32485                [New Mexico, Albuquerque, New Mexico]
32486    [Oakland County, West Bloomfield, Minneapolis,...
32487                                                   []
32488                                                   []
Name: NER, Length: 32489, dtype: object

In [37]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [38]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [39]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,96623207231_10158378197842232,Facebook,2020-06-01 22:49:27,2021-08-31 16:30:44,link,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,WATCH LIVE: President Trump is addressing unre...,1.0,http://on.nbc10.com/b4bKlRO,https://www.facebook.com/96623207231/posts/101...,691512,-1.134969,1.0,35,4,77,15,0,3,1,28,0,1,33,19,61,10,8,15,14,25,0,4,3535,NBC10 Philadelphia,nbcphiladelphia,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NBC10 Philadelphia is the place to go for excl...,True,2009-06-29 01:05:54,en,0,3535|10158378197842232,[],[],[],[]
1,1,1265297223574474_2731921486912033,Facebook,2020-06-01 22:49:25,2021-04-14 09:17:59,photo,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,UPDATE: Family Memorial Service for George Flo...,1.0,https://www.facebook.com/ThePressRelease4u/pho...,https://www.facebook.com/1265297223574474/post...,477,-24.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,1,0,1,2,0,0,5972645,The Press Release with Portia Bright,ThePressRelease4u,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Live interviews and real news facts all consum...,False,2017-10-15 01:53:17,en,0,5972645|2731921486912033,[],[],[],[]
2,2,100395078149910_161877328668351,Facebook,2020-06-01 22:49:24,2021-06-18 22:28:53,photo,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,How many of you were at Saturday’s event “Stan...,1.0,https://www.facebook.com/Topekaunited/photos/a...,https://www.facebook.com/100395078149910/posts...,0,4.736842,1.0,48,8,3,30,0,1,0,0,0,7,4,4,2,4,0,0,0,0,0,1,12642648,Topeka United,Topekaunited,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,We are a group of individuals who came togethe...,False,2019-12-25 15:29:15,en,0,12642648|161877328668351,[],[],[],[]
3,3,762995300426539_3094901393902573,Facebook,2020-06-01 22:49:24,2021-06-13 14:30:17,live_video_complete,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,LIVE: @NBCNews Special Report: President Trump...,RIGHT NOW:,2.0,https://www.facebook.com/NBCNews/videos/130791...,https://www.facebook.com/762995300426539/posts...,3153,-5.666667,2.0,7,1,1,1,0,1,1,0,0,0,31,4,17,9,2,1,4,0,0,0,2186839,Shannon Miller,Topekaunited,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Weekend AM Anchor/Reporter for NBC Connecticut...,False,2014-10-17 16:10:07,en,0,2186839|3094901393902573,[],[],[],[]
4,4,33957807912_10157309663472913,Facebook,2020-06-01 22:49:22,2021-05-17 12:46:14,status,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,LIVE: @NBCNews Special Report: President Trump...,What we witnessed happening to George Floyd ha...,2.0,https://www.facebook.com/NBCNews/videos/130791...,https://www.facebook.com/33957807912/posts/101...,2353,1.323529,2.0,27,2,7,9,0,0,0,0,0,0,18,2,5,9,0,0,0,0,0,0,8050974,Lamplighters Music Theatre,Lamplighters,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,http://lamplighters.org/about/reviews.html h...,False,2008-07-17 18:14:18,en,0,8050974|10157309663472913,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [40]:
ct_ner['NER-1']=ct_ner['NER']

In [41]:
ct_ner['NER-1'][0]

[]

In [42]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [43]:
ct_ner[:100]

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,96623207231_10158378197842232,Facebook,2020-06-01 22:49:27,2021-08-31 16:30:44,link,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,WATCH LIVE: President Trump is addressing unre...,1.0,http://on.nbc10.com/b4bKlRO,https://www.facebook.com/96623207231/posts/101...,691512,-1.134969,1.0,35,4,77,15,0,3,1,28,0,1,33,19,61,10,8,15,14,25,0,4,3535,NBC10 Philadelphia,nbcphiladelphia,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NBC10 Philadelphia is the place to go for excl...,True,2009-06-29 01:05:54,en,0,3535|10158378197842232,[],[],[],[],[]
1,1,1265297223574474_2731921486912033,Facebook,2020-06-01 22:49:25,2021-04-14 09:17:59,photo,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,UPDATE: Family Memorial Service for George Flo...,1.0,https://www.facebook.com/ThePressRelease4u/pho...,https://www.facebook.com/1265297223574474/post...,477,-24.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,1,0,1,2,0,0,5972645,The Press Release with Portia Bright,ThePressRelease4u,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Live interviews and real news facts all consum...,False,2017-10-15 01:53:17,en,0,5972645|2731921486912033,[],[],[],[],[]
2,2,100395078149910_161877328668351,Facebook,2020-06-01 22:49:24,2021-06-18 22:28:53,photo,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,How many of you were at Saturday’s event “Stan...,1.0,https://www.facebook.com/Topekaunited/photos/a...,https://www.facebook.com/100395078149910/posts...,0,4.736842,1.0,48,8,3,30,0,1,0,0,0,7,4,4,2,4,0,0,0,0,0,1,12642648,Topeka United,Topekaunited,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,We are a group of individuals who came togethe...,False,2019-12-25 15:29:15,en,0,12642648|161877328668351,[],[],[],[],[]
3,3,762995300426539_3094901393902573,Facebook,2020-06-01 22:49:24,2021-06-13 14:30:17,live_video_complete,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,LIVE: @NBCNews Special Report: President Trump...,RIGHT NOW:,2.0,https://www.facebook.com/NBCNews/videos/130791...,https://www.facebook.com/762995300426539/posts...,3153,-5.666667,2.0,7,1,1,1,0,1,1,0,0,0,31,4,17,9,2,1,4,0,0,0,2186839,Shannon Miller,Topekaunited,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Weekend AM Anchor/Reporter for NBC Connecticut...,False,2014-10-17 16:10:07,en,0,2186839|3094901393902573,[],[],[],[],[]
4,4,33957807912_10157309663472913,Facebook,2020-06-01 22:49:22,2021-05-17 12:46:14,status,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,LIVE: @NBCNews Special Report: President Trump...,What we witnessed happening to George Floyd ha...,2.0,https://www.facebook.com/NBCNews/videos/130791...,https://www.facebook.com/33957807912/posts/101...,2353,1.323529,2.0,27,2,7,9,0,0,0,0,0,0,18,2,5,9,0,0,0,0,0,0,8050974,Lamplighters Music Theatre,Lamplighters,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,http://lamplighters.org/about/reviews.html h...,False,2008-07-17 18:14:18,en,0,8050974|10157309663472913,[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [44]:
%cd mapping_US

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_US


In [45]:
!ls

 Country.txt			 dataframe0610_post.csv
 dataframe0529_post.csv		 dataframe0610_post_remove.csv
 dataframe0529_post_remove.csv	 dataframe0611_post.csv
 dataframe0530_post.csv		 dataframe0611_post_remove.csv
 dataframe0530_post_remove.csv	 dataframe_mapping
 dataframe0531_post.csv		 dataframe_mapping_0529.csv
 dataframe0531_post_remove.csv	 dataframe_mapping_0530.csv
 dataframe0601_post.csv		 dataframe_mapping_0531.csv
 dataframe0601_post_remove.csv	 dataframe_mapping_0601.csv
 dataframe0602_post.csv		 dataframe_mapping_0602.csv
 dataframe0602_post_remove.csv	 dataframe_mapping_0603.csv
 dataframe0603_post.csv		 dataframe_mapping_0604.csv
 dataframe0603_post_remove.csv	 dataframe_mapping_0605.csv
 dataframe0604_post.csv		 dataframe_mapping_0606.csv
 dataframe0604_post_remove.csv	 dataframe_mapping_0607.csv
 dataframe0605_post.csv		 dataframe_mapping_0608.csv
 dataframe0605_post_remove.csv	 dataframe_mapping_0609.csv
 dataframe0606_post.csv		 dataframe_mapping_0610.csv
 dataframe0606_post

In [46]:
ct_mapping=pd.read_csv('dataframe_mapping_0601.csv',encoding='ISO-8859-1')

In [47]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,8856,"Minneapolis, Hennepin County, Minnesota, 55488...",44.977300,-93.265469
1,1,minnesota,2579,"Minnesota, United States",45.989659,-94.611329
2,2,america,2183,"America, Horst aan de Maas, Limburg, Nederland",51.437148,5.979900
3,3,u.s.,1792,"Unterer See, Bahnhof, BÃ¶blingen (Stadt), BÃ¶b...",48.684067,9.009582
4,4,the united states,1658,"The Summit, Guadalupe County, Texas, United St...",29.633008,-97.988594
...,...,...,...,...,...,...
3058,3058,truah,1,"Truah, Boe & Quilla, Nimba County, Liberia",6.775220,-8.525260
3059,3059,keshet,1,"×§×©×ª, ×××¢×¦× ××××¨××ª ××××, × ×¤...",32.980733,35.808586
3060,3060,gambrills,1,"Gambrills, Anne Arundel County, Maryland, Unit...",39.096349,-76.654910
3061,3061,paz,1,"Paz, OpÄina Cerovlje, Istarska Å¾upanija, 524...",45.290837,14.094491


In [48]:
country_list=pd.read_csv('Country.txt')
country_list['Name']=country_list['Name'].str.lower()

In [49]:
country_list['Name']

0            afghanistan
1          �land islands
2                albania
3                algeria
4         american samoa
             ...        
250                   us
251    the united states
252              america
253                  u.s
254        united states
Name: Name, Length: 255, dtype: object

In [50]:
states_list=pd.read_csv('states.csv')
states_list['State']

0                             Alabama, United States
1                              Alaska, United States
2                             Arizona, United States
3                            Arkansas, United States
4                          California, United States
                           ...                      
165         Armagh, Northern Ireland, United Kingdom
166           Down, Northern Ireland, United Kingdom
167      Fermanagh, Northern Ireland, United Kingdom
168    Londonderry, Northern Ireland, United Kingdom
169         Tyrone, Northern Ireland, United Kingdom
Name: State, Length: 170, dtype: object

In [51]:
from numpy.core.numeric import NaN
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['City'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [52]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(states_list['State'])):
    if ct_mapping['Mapping'][i]==states_list['State'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [53]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['Mapping'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

In [54]:
pd.set_option('display.max_rows', None)

In [55]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,8856,"Minneapolis, Hennepin County, Minnesota, 55488...",44.977300,-93.265469
1,1,minnesota,2579,NaN,45.989659,-94.611329
2,2,america,2183,NaN,51.437148,5.979900
3,3,u.s.,1792,NaN,48.684067,9.009582
4,4,the united states,1658,NaN,29.633008,-97.988594
5,5,us,1291,NaN,39.783730,-100.445882
6,6,washington,639,"Washington, District of Columbia, United States",38.895037,-77.036543
7,7,chicago,551,"Chicago, Cook County, Illinois, United States",41.875562,-87.624421
8,8,california,439,NaN,36.701463,-118.755997
9,9,atlanta,438,"Atlanta, Fulton County, Georgia, 30303-3506, U...",33.748992,-84.390264


In [56]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,8856,"Minneapolis, Hennepin County, Minnesota, 55488...",44.977300,-93.265469
1,1,minnesota,2579,NaN,45.989659,-94.611329
2,2,america,2183,NaN,51.437148,5.979900
3,3,u.s.,1792,NaN,48.684067,9.009582
4,4,the united states,1658,NaN,29.633008,-97.988594
5,5,us,1291,NaN,39.783730,-100.445882
6,6,washington,639,"Washington, District of Columbia, United States",38.895037,-77.036543
7,7,chicago,551,"Chicago, Cook County, Illinois, United States",41.875562,-87.624421
8,8,california,439,NaN,36.701463,-118.755997
9,9,atlanta,438,"Atlanta, Fulton County, Georgia, 30303-3506, U...",33.748992,-84.390264


In [57]:
ct_mapping.to_csv('dataframe_mapping_0601.csv')

In [58]:
#ct_mapping=pd.read_csv('dataframe_mapping_0530.csv',encoding='ISO-8859-1')

In [59]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [60]:
ct_ner[:100]

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,96623207231_10158378197842232,Facebook,2020-06-01 22:49:27,2021-08-31 16:30:44,link,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,WATCH LIVE: President Trump is addressing unre...,1.0,http://on.nbc10.com/b4bKlRO,https://www.facebook.com/96623207231/posts/101...,691512,-1.134969,1.0,35,4,77,15,0,3,1,28,0,1,33,19,61,10,8,15,14,25,0,4,3535,NBC10 Philadelphia,nbcphiladelphia,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NBC10 Philadelphia is the place to go for excl...,True,2009-06-29 01:05:54,en,0,3535|10158378197842232,[],[],[],[],[]
1,1,1265297223574474_2731921486912033,Facebook,2020-06-01 22:49:25,2021-04-14 09:17:59,photo,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,UPDATE: Family Memorial Service for George Flo...,1.0,https://www.facebook.com/ThePressRelease4u/pho...,https://www.facebook.com/1265297223574474/post...,477,-24.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,1,0,1,2,0,0,5972645,The Press Release with Portia Bright,ThePressRelease4u,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Live interviews and real news facts all consum...,False,2017-10-15 01:53:17,en,0,5972645|2731921486912033,[],[],[],[],[]
2,2,100395078149910_161877328668351,Facebook,2020-06-01 22:49:24,2021-06-18 22:28:53,photo,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,NaN,How many of you were at Saturday’s event “Stan...,1.0,https://www.facebook.com/Topekaunited/photos/a...,https://www.facebook.com/100395078149910/posts...,0,4.736842,1.0,48,8,3,30,0,1,0,0,0,7,4,4,2,4,0,0,0,0,0,1,12642648,Topeka United,Topekaunited,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,We are a group of individuals who came togethe...,False,2019-12-25 15:29:15,en,0,12642648|161877328668351,[],[],[],[],[]
3,3,762995300426539_3094901393902573,Facebook,2020-06-01 22:49:24,2021-06-13 14:30:17,live_video_complete,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,LIVE: @NBCNews Special Report: President Trump...,RIGHT NOW:,2.0,https://www.facebook.com/NBCNews/videos/130791...,https://www.facebook.com/762995300426539/posts...,3153,-5.666667,2.0,7,1,1,1,0,1,1,0,0,0,31,4,17,9,2,1,4,0,0,0,2186839,Shannon Miller,Topekaunited,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Weekend AM Anchor/Reporter for NBC Connecticut...,False,2014-10-17 16:10:07,en,0,2186839|3094901393902573,[],[],[],[],[]
4,4,33957807912_10157309663472913,Facebook,2020-06-01 22:49:22,2021-05-17 12:46:14,status,WATCH LIVE: President Trump Addresses National...,nbcphiladelphia.com,LIVE: @NBCNews Special Report: President Trump...,What we witnessed happening to George Floyd ha...,2.0,https://www.facebook.com/NBCNews/videos/130791...,https://www.facebook.com/33957807912/posts/101...,2353,1.323529,2.0,27,2,7,9,0,0,0,0,0,0,18,2,5,9,0,0,0,0,0,0,8050974,Lamplighters Music Theatre,Lamplighters,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,http://lamplighters.org/about/reviews.html h...,False,2008-07-17 18:14:18,en,0,8050974|10157309663472913,[],[],[],[],[]
5,5,123839200978190_3462861730409237,Facebook,2020-06-01 22:49:22,2021-07-31 06:22:30,status,WATCH LIVE: President Trump Addresses National

In [61]:
ct_ner['NER-1'][:100]

0                                                    []
1                                                    []
2                                                    []
3                                                    []
4                                                    []
5                                                    []
6     [NOLA New Orleans Style BBQ Restaurant., Baran...
7     [nan, nan, Jackson, Hinds County, Mississippi,...
8     [nan, nan, Jackson, Hinds County, Mississippi,...
9     [nan, nan, Jackson, Hinds County, Mississippi,...
10    [Minneapolis, Hennepin County, Minnesota, 5548...
11    [Memphis, Shelby County, Tennessee, 3813, Unit...
12    [Los Angeles, Los Angeles County, California, ...
13    [Niagara Falls, New York, Bath Avenue, Niagara...
14    [Sacramento, Sacramento County, California, 95...
15    [Harrisonburg, Virginia, United States, Niagar...
16    [Harrisonburg, Virginia, United States, Niagar...
17    [Riverside, The Willows, Deeping Gate, Cit

In [62]:
ct_ner.to_csv('dataframe0601_post_remove.csv')

In [63]:
!ls

 Country.txt			 dataframe0610_post.csv
 dataframe0529_post.csv		 dataframe0610_post_remove.csv
 dataframe0529_post_remove.csv	 dataframe0611_post.csv
 dataframe0530_post.csv		 dataframe0611_post_remove.csv
 dataframe0530_post_remove.csv	 dataframe_mapping
 dataframe0531_post.csv		 dataframe_mapping_0529.csv
 dataframe0531_post_remove.csv	 dataframe_mapping_0530.csv
 dataframe0601_post.csv		 dataframe_mapping_0531.csv
 dataframe0601_post_remove.csv	 dataframe_mapping_0601.csv
 dataframe0602_post.csv		 dataframe_mapping_0602.csv
 dataframe0602_post_remove.csv	 dataframe_mapping_0603.csv
 dataframe0603_post.csv		 dataframe_mapping_0604.csv
 dataframe0603_post_remove.csv	 dataframe_mapping_0605.csv
 dataframe0604_post.csv		 dataframe_mapping_0606.csv
 dataframe0604_post_remove.csv	 dataframe_mapping_0607.csv
 dataframe0605_post.csv		 dataframe_mapping_0608.csv
 dataframe0605_post_remove.csv	 dataframe_mapping_0609.csv
 dataframe0606_post.csv		 dataframe_mapping_0610.csv
 dataframe0606_post